<a href="https://colab.research.google.com/github/niklashaffert/LandUseClassification/blob/main/01_data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>